# YOLO v1

[paper](https://arxiv.org/pdf/1506.02640.pdf)

Key points:

- S x S grid. S = 7
- predicts B boxes for each cell. B = 2
- Responsible cell:
    - the cell that contains bbox midpoint.
    - Among B predicted boxes, only the one that has highest IoU will be responsible.
- predicts confidence each cell. confidence = IoU
- predicts x, y, w, h each cell:
    - x, y: they are midpoint coordinates relative to cell origin, h, w.
        Meaning, cell h, w are 1, 1, and x, y will be in [0, 1]
    - h, w: they are bbox height, width relative to whole image.
- predicts C classes each cell.
- All are trained only when the cell is responsible for a bbox.
- Each cell can only predict 1 object. although it tries to predict B bboxes
- Predicted tensor is of shape [S, S, (C + 5B)]
- Architecture is simply a CNN followed by a flatten and fully-connected layers.
- While inference, multiply C probabilities with predicted confidence.
- While inference, apply NMS
- All losses are MSE variations.

Hyperparams:
- leaky relu
- batch size 64
- epochs 135 (with pre-trained)
- momentum 0.9
- decay: 0.0005
- lr:
    - 10^-3 for few epochs.
    - 10^-2 for +75 epochs
    - 10^-3 for +30 epochs.
    - 10^-4 for +30 epochs.
- Extensive augmentation:
    - Random scaling and translation up to 20%
    - randomly adjust the exposure and saturation of the image by up to a factor of 1.5 in the HSV color space.
- dropout of 0.5 on last fully-connected
-


In [ ]:
! pip install pytorch-lightning

In [ ]:
import torch
import torch.nn as nn
from typing import List, Union
import pytorch_lightning as pl

In [ ]:
"""
Information about architecture config:
- Tuple is structured by (kernel_size, filters, stride, padding)
- "M" is simply maxpooling with stride 2x2 and kernel 2x2
- List is structured by tuples and lastly int with number of repeats
"""

architecture_config = [
    (7, 64, 2, 3),
    "M",
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1),
]

In [ ]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(CNNBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.leakyrelu = nn.LeakyReLU(0.1)

    def forward(self, x):
        return self.leakyrelu(self.batchnorm(self.conv(x)))


class SimpleCNN(nn.Module):
    def __init__(
        self,
        architecture: List[Union[tuple, str, list]],
        in_channels: int,
    ):
        super(SimpleCNN, self).__init__()
        layers = []
        for module in architecture:
            if type(module) is tuple:
                layers.append(self._get_cnn_block(module, in_channels))
                in_channels = module[1]
            elif module == 'M':
                layers.append(nn.MaxPool2d(
                    kernel_size=(2, 2),
                    stride=(2, 2),
                ))
            elif type(module) is list:
                for i in range(module[-1]):
                    for j in range(len(module) - 1):
                        layers.append(self._get_cnn_block(module[j], in_channels))
                        in_channels = module[j][1]
        self.model = nn.Sequential(*layers)

    @staticmethod
    def _get_cnn_block(module: tuple, in_channels):
        kernel_size, filters, stride, padding = module
        return CNNBlock(
            in_channels,
            filters,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

    def forward(self, x):
        return self.model(x)


class YoloV1(nn.Module):
    def __init__(self, in_channels, split_size, num_boxes, num_classes):
        super(YoloV1, self).__init__()
        self.darknet = SimpleCNN(architecture_config, in_channels)

        S, B, C = split_size, num_boxes, num_classes
        self.fcs = nn.Sequential(
            nn.Flatten(),

            nn.Linear(1024 * S * S, 496),
            nn.Dropout(0.1),
            nn.LeakyReLU(0.1),

            nn.Linear(496, S * S * (C + B * 5)),
        )
        self.final_shape = (-1, S, S, (C + B * 5))

    def forward(self, x):
        x = self.darknet(x)
        out = self.fcs(torch.flatten(x, start_dim=1))
        out = out.view(self.final_shape)
        return out

In [ ]:
model = YoloV1(in_channels=3, split_size=7, num_boxes=2, num_classes=21)
random_batch = torch.rand((2, 3, 448, 448))
random_output = model(random_batch)
random_output.shape

In [ ]:
class YoloV1Loss(nn.Module):
    def __init__(self, num_boxes: int, num_classes: int):
        super().__init__()

        self.num_boxes = num_boxes
        self.num_classes = num_classes

    def forward(self, pred, true):
        pass


In [ ]:
class YoloV1PL(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.yolo_v1 = YoloV1(
            in_channels=3,
            split_size=7,
            num_boxes=2,
            num_classes=21
        )

    def forward(self, x):
        return self.yolo_v1(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log('val_loss', loss)